<a href="https://colab.research.google.com/github/luana-pgm/Quinto-Projeto-Analise-de-Dados-de-Sentimentos-de-filmes/blob/main/Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Instalar PySpark
!pip -q install pyspark==3.5.1

# Montar o Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Caminho base (ajuste se o nome da pasta for diferente)
base_path = "/content/drive/MyDrive/Eixo_05 (1)/dados/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("imdb-classificacao")
    .getOrCreate()
)

spark

In [ ]:
# Carregar dados TF-IDF
df = spark.read.parquet(base_path + "TFIDFfeaturizedData")

# Verificar estrutura
df.printSchema()
df.show(5)

root
 |-- sentiment: string (nullable = true)
 |-- review: string (nullable = true)
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)

+---------+--------------------+-----+--------------------+
|sentiment|              review|label|            features|
+---------+--------------------+-----+--------------------+
| negative|i rented i am cur...|  0.0|(250,[3,13,16,24,...|
| negative|i am curious yell...|  0.0|(250,[1,3,6,11,12...|
| negative|if only to avoid ...|  0.0|(250,[3,26,61,77,...|
| negative|this film was pro...|  0.0|(250,[3,47,70,100...|
| negative|oh brotherafter h...|  0.0|(250,[3,5,10,46,4...|
+---------+--------------------+-----+--------------------+
only showing top 5 rows



In [ ]:
train_df, test_df = df.randomSplit([0.8, 0.2], seed=42)
print(f"Treino: {train_df.count()} | Teste: {test_df.count()}")

Treino: 40099 | Teste: 9901


In [ ]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(
    featuresCol="features",
    labelCol="label",
    maxIter=10
)

lr_model = lr.fit(train_df)
print("✅ Modelo treinado com sucesso!")

✅ Modelo treinado com sucesso!


In [ ]:
pred = lr_model.transform(test_df)
pred.select("sentiment", "label", "prediction", "probability").show(10, truncate=False)

+---------+-----+----------+----------------------------------------+
|sentiment|label|prediction|probability                             |
+---------+-----+----------+----------------------------------------+
|negative |0.0  |0.0       |[0.5060994150932712,0.49390058490672883]|
|negative |0.0  |0.0       |[0.5849408445674472,0.4150591554325528] |
|negative |0.0  |0.0       |[0.5240827448331777,0.47591725516682226]|
|negative |0.0  |0.0       |[0.7812352785206714,0.21876472147932857]|
|negative |0.0  |0.0       |[0.7538377804013703,0.24616221959862972]|
|negative |0.0  |0.0       |[0.6839306954473406,0.31606930455265936]|
|negative |0.0  |0.0       |[0.6483856033303106,0.3516143966696894] |
|negative |0.0  |0.0       |[0.5356252250157527,0.4643747749842473] |
|negative |0.0  |0.0       |[0.7512120305751812,0.24878796942481884]|
|negative |0.0  |0.0       |[0.7601679004643271,0.23983209953567286]|
+---------+-----+----------+----------------------------------------+
only showing top 10 

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

# AUC - qualidade da separação entre positivos e negativos
auc_evaluator = BinaryClassificationEvaluator()
auc = auc_evaluator.evaluate(pred)

# Acurácia - percentual de acertos
acc_evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
acc = acc_evaluator.evaluate(pred)

print(f"AUC: {auc:.4f}")
print(f"Acurácia: {acc:.4f}")

AUC: 0.7660
Acurácia: 0.7000


In [ ]:
lr_model.save(base_path + "modelo_logistic_regression")
print("Modelo salvo em:", base_path + "modelo_logistic_regression")

Modelo salvo em: /content/drive/MyDrive/Eixo_05 (1)/dados/modelo_logistic_regression
